In [1]:
import asyncio
import re
import json
import pandas as pd

from google.adk.agents import LlmAgent
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.models.google_llm import Gemini
from google.genai import types
from google.genai.types import Content, Part

from dotenv import load_dotenv
load_dotenv()
import json

from planner_agent import run_planner, extract_json
from executor_tool import execute_query_plan

In [6]:
session_service = InMemorySessionService()
APP_NAME = "migration agentic model testing"
USER_ID = "Yi"
SESSION_ID = "test_session_1"

await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID,
)

question = "Find the out-migration rates of Midwest households whose gross income is over $100,000, from 2015 to 2022"

plan = await run_planner(
    question=question,
    session_service=session_service,
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID,
)

print("parsed plan: \n", plan)

parsed plan: 
 {'intent': 'Compute out-migration rates for Midwest households with gross income over $100,000, from 2015 to 2022.', 'datasets': [{'name': 'soi_migration_long', 'alias': 's', 'source': 'local_file', 'path': 'data/processed/soi_migration_long.csv'}, {'name': 'statefips_dict', 'alias': 'f', 'source': 'local_file', 'path': 'data/reference/statefips_dict.csv'}], 'joins': [{'type': 'inner', 'left_dataset': 's', 'right_dataset': 'f', 'on': [{'left_column': 'state', 'right_column': 'state'}]}], 'filters': {'year': {'between': [2015, 2022]}, 'agi_stub': {'in': [6, 7]}, 'age_class': {'equals': 0}, 'region': {'equals': 'Midwest'}}, 'group_by': ['year'], 'metrics': [{'name': 'outflow_n1', 'agg': 'sum', 'column': 'n1', 'filter': {'class': 'outflow'}}, {'name': 'total_n1', 'agg': 'sum', 'column': 'n1', 'filter': {'class': 'total'}}], 'derived_columns': [{'name': 'out_migration_rate', 'expression': 'outflow_n1 / total_n1'}], 'time': {'column': 'year', 'granularity': 'year', 'range': {

In [7]:
def load_executor_prompt() -> str:
    with open("prompts/query_executor.txt", "r", encoding="utf-8") as f:
        return f.read()
    
instruction = load_executor_prompt()

def build_executor_agent():
    instruction_text = load_executor_prompt()

    executor_agent = LlmAgent(
        name="query_executor",
        instruction=instruction_text,          
        model=Gemini(
            model = "gemini-2.5-flash") ,      
        tools=[execute_query_plan],           
    )
    return executor_agent

executor_agent = build_executor_agent()

runner = InMemoryRunner(agent=executor_agent, app_name=APP_NAME)
session_service = runner.session_service

await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID,
)

text = "QUERY_PLAN:\n" + json.dumps(plan)
content = types.Content(
    role="user",
    parts=[types.Part(text=text)]
)

events = runner.run(
    user_id=USER_ID,
    session_id=SESSION_ID,
    new_message=content,
)

final_text = None

for event in events:
    if hasattr(event, "is_final_response") and event.is_final_response():
        if hasattr(event, "output_text") and event.output_text is not None:
            final_text = event.output_text
        else:
            final_text = event.content.parts[0].text

# Extract JSON array from the final text
try:
    match = re.search(r"\[.*\]", final_text, flags=re.S)

except Exception as e:
    raise ValueError("Error while searching for JSON array in model output") from e
if not match:
    raise ValueError("Could not find JSON array in model output")

json_text = match.group(0)
rows = json.loads(json_text)
df = pd.DataFrame(rows)

In [8]:
print(df)

   out_migration_rate  outflow_n1  total_n1  year
0            0.012912       59499   4607888  2015
1            0.021449      104383   4866599  2016
2            0.030015      149216   4971447  2017
3            0.020487      108555   5298804  2018
4            0.019913      113542   5702003  2019
5            0.021790      128752   5908717  2020
6            0.022981      135969   5916469  2021
7            0.022813      154419   6768836  2022


From our experiment, executor + planner is capable of one SQL query and is very stable. We should consider using this combination for initial data pulling for later analysis.